# Noisy Labels in Dataset (voxceleb)

dataset의 labelling이 잘못 되었을 떄의  효과를 보기 위해.

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
voxc1_si = pd.read_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1_si.csv")

In [3]:
# split voxc1_si into in_dist and out_dist
voxc1_si_spks = voxc1_si.spk.unique().tolist()
even_num = len(voxc1_si_spks)//2
in_spks = np.random.choice(voxc1_si_spks, size=even_num, replace=False) 
out_spks = np.random.choice(list(set(voxc1_si_spks) - set(in_spks)), size=even_num, replace=False)

In [4]:
voxc1_si_in = voxc1_si[voxc1_si.spk.isin(in_spks)]
voxc1_si_out = voxc1_si[voxc1_si.spk.isin(out_spks)]

voxc1_si_in.loc[:, 'label'] = voxc1_si_in.groupby("spk").ngroup()
voxc1_si_out.loc[:, 'label'] = voxc1_si_out.groupby("spk").ngroup()

/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [5]:
voxc1_si_in.to_csv("/dataset/SV_sets/voxceleb1/dataframes/noisy_train/voxc1_si_in.csv", index=False)

## out-dist injection

In [20]:
injected_out = voxc1_si_out.groupby("label", group_keys=False).apply(lambda x: x.sample(frac=0.2))
# injected noise are used for train set.
injected_out.loc[:, 'set'] = 'train'

In [21]:
# out-dist injected
out_injected_si = pd.concat([voxc1_si_in, injected_out])
out_injected_si = out_injected_si.sort_values('label')

In [22]:
out_injected_si.to_csv("/dataset/SV_sets/voxceleb1/dataframes/noisy_train/voxc1_injected_si.csv", index=False)

## in-dist random labels

In [15]:
# in-dist label random shuffle
seleted_samples = voxc1_si_in.groupby("label", group_keys=False).apply(lambda x: x.sample(frac=0.2))

In [16]:
label_dist = voxc1_si_in.label.value_counts().values
label_dist_norm = label_dist / label_dist.sum()

In [17]:
random_label = np.random.randint(0, even_num, size=len(seleted_samples))

In [18]:
random_label_si = voxc1_si_in.copy()
random_label_si.loc[seleted_samples.index, 'label'] = random_label

In [19]:
random_label_si.to_csv("/dataset/SV_sets/voxceleb1/dataframes/noisy_train/voxc1_randomLabel_si.csv", index=False)